In [1]:
from src.datasets.data_module import CloudCoverDataModule
from pathlib import Path
from src.models.deeplab import LightningDeeplab
from src.training.trainer import train
from src.testing.tester import test

In [2]:
import torch
print(torch.__version__)
cuda_available = torch.cuda.is_available()
print("CUDA GPU available:", cuda_available)

2.1.1+cu121
CUDA GPU available: True


### Dataloader

In [3]:
data_module = CloudCoverDataModule(
    train_X_folder_path=Path("../data/final/public/train_features/"),
    train_y_folder_path=Path("../data/final/public/train_labels/"),
    test_X_folder_path=Path("../data/final/private/test_features/"),
    test_y_folder_path=Path("../data/final/private/test_labels/"),
    train_batch_size=4,
    val_batch_size=8,
    test_batch_size=8,
    val_size=0.2,
    random_state=42
)

In [4]:
data_module.prepare_data()

In [5]:
data_module.setup(stage="fit")
data_module.setup(stage="test")

### Train

In [6]:
deeplab = train(
    model=LightningDeeplab(n_channels=4, n_classes=2, bilinear=True, learning_rate=0.001),
    run_name="deeplab",
    model_version=0,
    data_module=data_module,
    max_epochs=2,
    patience=5
)

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory logs\deeplab\version_0\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | model          | DeepLabV3          | 43.2 M
1 | train_jaccard  | BinaryJaccardIndex | 0     
2 | train_accuracy | BinaryAccuracy     | 0     
3 | val_jaccard    | BinaryJaccardIndex | 0     
4 | val_accuracy   | BinaryAccuracy     | 0     
5 | test_jaccard   | BinaryJaccardIndex | 0     
6 | test_accuracy  | BinaryAccuracy     | 0     
------------------------------------------------------
43.2 M    Trainable params
0         Non-trainable params
43.2 M    Total params
172.958   Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=2` reached.


### Test  
Replace the checkpoint path with the best checkpoint from the training.

In [7]:
#deeplab = LightningDeeplab.load_from_checkpoint(checkpoint_path='./logs/deeplab/version_0/checkpoints/deeplab-epoch=01-val_loss=0.67.ckpt', n_channels=4, n_classes=2)

In [8]:
test(
    model=deeplab,
    run_name="deeplab",
    model_version=0,
    data_module=data_module
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_epoch      0.6116716861724854
   test_jaccard_epoch       0.6116716861724854
     test_loss_epoch        0.6771873831748962
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
